# Sentinel-5P OFFL L3_SO2简介与Notebook示例
该数据集提供Offline的大气二氧化硫（SO2）浓度信息。
二氧化硫 (SO2) 通过自然和人为过程进入地球大气层，可在局部和全球范围内发生化学反应，会造成短期的环境污染和长期的气候变化。统计表明，只有约30%的SO2排放来自自然界，其余大部分均由人类活动造成。SO2排放对人类健康和空气质量会产生不利影响，同时由SO2生成的硫酸盐气溶胶会影响太阳辐射，进而影响气候变化。Sentinel-5P/TROPOMI可以以1天的重访周期和3.5 x 5.5 km2的空间分辨率连续监测对流层SO2浓度情况。<a href="http://www.tropomi.eu/data-products/sulphur-dioxide" target="_blank">【更多信息】</a>

## OFFL/L3 产品

Sentinel-5P Level 2使用netCDF格式存储数据，并没有直接映射经纬度信息。因此，AI Earth平台将Level 2级数据转换为Level 3级数据，保证影像正确发布。Level 3级产品生产步骤如下：

1. 基于Level 2级产品数据，读取latitude_bounds和longitude_bounds两个图层，获取每一个有效栅格单元（Cell）的地理范围；
2. 设定Level 3级产品的分辨率为0.02度，基于步骤1中获取的有效地理范围，使用反距离权重（IDW）进行插值；
3. 导出部分图层。

由于Level 2使用qa_value表征产品质量，因此Level 3级数据产品仅保留高质量部分数据，筛选条件如下：

1. snow_cover < 0.5
2. sea_ice_cover < 0.5
3. sulfurdioxide_total_air_mass_factor_polluted > 0.1
4. sulfurdioxide_total_vertical_column > -0.001 mol/m^2.
5. qa_value > 0.5
6. cloud_fraction_crb < 0.3
7. solar_zenith_angle < 60°

需注意qa_value仅用于sulfurdioxide_total_vertical_column筛选。同时，由于Level 2中包含不同高度（1km，7km和15km）的SO2柱浓度数据sulfurdioxide_total_vertical_column_{1,7,15}km，一定程度上能反应火山喷发引发的SO2排放，Level 3仅保留solar_zenith_angle<70部分数据。

## 数据集检索

In [ ]:
import aie
aie.Authenticate()
aie.Initialize()


dataset = aie.ImageCollection('SENTINEL5P_OFFL_L3_SO2') \
             .filterDate('2022-10-01', '2022-10-31') \
             .limit(1)

map = aie.Map(
    center=dataset.getCenter(),
    height=800,
    zoom=1
)

vis_params = {
    'bands': ['SO2_column_number_density'],
    'min': 0,
    'max': 0.0005,
    'palette': ['#000000', '#0000FF', '#800080', 
                '#00FFFF', '#008000', '#FFFF00', '#FF0000'],
}

map.addLayer(
    dataset,
    vis_params,
    'SENTINEL5P_OFFL_L3__SO2___',
    bounds=dataset.getBounds()
)
map

## 单景影像检索

In [ ]:
import aie
aie.Authenticate()
aie.Initialize()

img = aie.Image('S5P_OFFL_L3__SO2____20210207T132947_20210207T151117_17212_01_020104_20210209T162121')

map = aie.Map(
    center=img.getCenter(),
    height=800,
    zoom=1
)

vis_params = {
    'bands': ['SO2_column_number_density'],
    'min': 0,
    'max': 0.0005,
    'palette': ['#000000', '#0000FF', '#800080', 
                '#00FFFF', '#008000', '#FFFF00', '#FF0000'],
}

map.addLayer(
    img,
    vis_params,
    'SENTINEL5P_OFFL_L3__SO2___',
    bounds=img.getBounds()
)
map